# Capstone Projekt Rossmann

# Feature Engineering

## Functions needed for testing

### Test models with test and tain data. Test includes the last 8 weeks from each store

In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from pandas.api.types import infer_dtype

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor

from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score
from math import sqrt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

pd.set_option('display.max_columns', None)

In [48]:
## Test models with test and tain data. Test includes the last 8 weeks from each store

def build_neural_network(X_train):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model





def testModelsTestSplit8W(df, scaler):
	train_data = []
	test_data = []

	# Group by store and split into training and test data
	amount_test_weeks = 8
	for store_id, group in df.groupby('Store'):
		train_data.append(group[: -amount_test_weeks])
		test_data.append(group[-amount_test_weeks:])

	# Combine the list entries to one dataframe
	train_df = pd.concat(train_data)
	test_df = pd.concat(test_data)

	X_train = train_df.drop(columns=['Future_Sales'])
	y_train = train_df['Future_Sales']
	X_test = test_df.drop(columns=['Future_Sales'])
	y_test = test_df['Future_Sales']

	# Scaling of the data
	if scaler:
		X_train = scaler.fit_transform(X_train)
		X_test = scaler.transform(X_test)

	def adj_r2_score(model, X, y):
		n = X.shape[0]
		p = X.shape[1]
		r2 = r2_score(y, model.predict(X))
		return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

	# Defining the models to test
	models = [
		('LinearRegression', LinearRegression(n_jobs=-1)),
		('XGBRegressor', XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3, learning_rate=0.1, n_jobs=-1, random_state=42, device="cuda")),
		#('GradientBoostingRegressor', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
  		('NeuralNetwork', KerasRegressor(build_fn=build_neural_network(X_train), epochs=100, batch_size=10, verbose=0)),
  		('MLPRegressor', MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200, shuffle=False, random_state=42))
		#('RidgeRegression', Ridge(random_state=42)),
		#('LassoRegression', Lasso(random_state=42)),
		#('DecisionTreeRegressor', DecisionTreeRegressor(random_state=42)),
		#('RandomForestRegressor', RandomForestRegressor(n_jobs=-1, max_depth=10, random_state=42, n_estimators=100)),
		#('SVR', SVR()),
		#('KNN', KNeighborsRegressor())
	]

	results = []
	# Train models and calculate metrics
	for name, model in models:
		model.fit(X_train, y_train)
		y_train_pred = model.predict(X_train)
		y_test_pred = model.predict(X_test)

		results.append({
			'Model': name,
			'RMSE_Train': sqrt(mse(y_train, y_train_pred)),
			'MAE_Train': mae(y_train, y_train_pred),
			'R2_Train': r2_score(y_train, y_train_pred),
			'Adj_R2_Train': adj_r2_score(model, X_train, y_train),
			'RMSE_Test': sqrt(mse(y_test, y_test_pred)),
			'MAE_Test': mae(y_test, y_test_pred),
			'R2_Test': r2_score(y_test, y_test_pred),
			'Adj_R2_Test': adj_r2_score(model, X_test, y_test)
		})
		#print last result
		print(results[-1])

	results_df = pd.DataFrame(results)
	return results_df

In [8]:
#!pip install scikeras


### Creates x splits in test and train where the last 8 weeks of each store are included in the respective test split and the splits are distributed evenly using gap

In [9]:
#Creates x splits in test and train where the last 8 weeks of each store are included in the respective test split and the splits are distributed
# evenly using gap

def testModelsCV8W(df, scaler):

    n_splits = 5
    window_size = 8
    total_weeks =109
    train_size = window_size / 0.2
    gap = int((total_weeks - window_size - train_size) // (n_splits))

    results = []

    for split in range(n_splits):
        train_data = []
        test_data = []

        for store_id, group in df.groupby('Store'):
            # calculate start and end index for test data
            if split == 0:
                test_start_index = -window_size
                test_df_store = group[test_start_index:] # No end index for the first split
            else:
                test_start_index = -(window_size + gap * split)
                test_end_index = test_start_index + window_size
                test_df_store = group[test_start_index:test_end_index]
                print("test:", test_df_store.shape, "Test Start Index:", test_start_index, "Test End Index:", test_end_index)

            train_start_index = -int(-test_start_index + gap + train_size)
            train_df_store = group[train_start_index:test_start_index]
            print("Train:", train_df_store.shape, "Train Start Index:", train_start_index, "Train End Index:", test_start_index)
            # Check if test set contains data
            if not test_df_store.empty:
                train_data.append(train_df_store)
                test_data.append(test_df_store)
            else:
                print(f"Store {store_id} has not enough data for splitting {split}")

        # Combine the list entries to one dataframe
        train_df_combined = pd.concat(train_data)
        test_df_combined = pd.concat(test_data)

        # Create feature and target data frames
        X_train = train_df_combined.drop(columns=['Future_Sales'])
        y_train = train_df_combined['Future_Sales']
        X_test = test_df_combined.drop(columns=['Future_Sales'])
        y_test = test_df_combined['Future_Sales']

        # Scaling of the data
        if scaler:
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

        def adj_r2_score(model, X, y):
            n = X.shape[0]
            p = X.shape[1]
            r2 = r2_score(y, model.predict(X))
            return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

    	# Defining the models to test
        models = [
            ('LinearRegression', LinearRegression(n_jobs=-1)),
            #('RidgeRegression', Ridge(random_state=42)),
            #('LassoRegression', Lasso(random_state=42)),
            #('DecisionTreeRegressor', DecisionTreeRegressor(random_state=42)),
            #('RandomForestRegressor', RandomForestRegressor(n_jobs=-1, max_depth=10, random_state=42, n_estimators=100)),
            #('SVR', SVR()),
            #('KNN', KNeighborsRegressor())
        ]

        # Train models and calculate metrics
        for name, model in models:
            model.fit(X_train, y_train)
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)

            results.append({
                'Model': name,
                'RMSE_Train': sqrt(mse(y_train, y_train_pred)),
                'MAE_Train': mae(y_train, y_train_pred),
                'R2_Train': r2_score(y_train, y_train_pred),
                'Adj_R2_Train': adj_r2_score(model, X_train, y_train),
                'RMSE_Test': sqrt(mse(y_test, y_test_pred)),
                'MAE_Test': mae(y_test, y_test_pred),
                'R2_Test': r2_score(y_test, y_test_pred),
                'Adj_R2_Test': adj_r2_score(model, X_test, y_test)
            })
            #print last result
            print(results[-1])

    results_df = pd.DataFrame(results)

    # calculate mean of all splits
    model_list = results_df['Model'].unique()
    # create resulte_mean_df
    resulte_mean_df = pd.DataFrame(columns=results_df.columns)
    # iterate over model_list
    for model in model_list:
        # get mean of each model
        mean = results_df[results_df['Model'] == model].mean(numeric_only=True)
        mean['Model'] = model
        # append mean to resulte_mean_df
        resulte_mean_df = pd.concat([resulte_mean_df, pd.DataFrame([mean], columns=results_df.columns)], ignore_index=True)

    return results_df, resulte_mean_df


## Feature Engineering

In [11]:
df = pd.read_csv('weekly_sales_with_store_info.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [12]:
print(df.info())
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150525 entries, 0 to 150524
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      150525 non-null  int64         
 1   Date                       150525 non-null  datetime64[ns]
 2   CW                         150525 non-null  int64         
 3   Month                      150525 non-null  int64         
 4   Year                       150525 non-null  int64         
 5   DayOfWeek                  150525 non-null  int64         
 6   Sales                      150525 non-null  int64         
 7   SalesPerCustomer           145809 non-null  float64       
 8   SalesPerOpenDay            145815 non-null  float64       
 9   Customers                  150525 non-null  int64         
 10  CustomersPerOpenDay        145815 non-null  float64       
 11  Open                       150525 non-null  int64   

,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,IsCompetition,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2Member,Promo2Active
99138,735,2013-12-08,49,12,2013,6,49876,12.003851,8312.666667,4155,692.500000,6,5,1,0,0,0,0,0,d,c,1920.0,4.0,2005.0,1,0,NaN,NaN,NaN,0,0
18684,139,2014-01-19,3,1,2014,6,29864,11.277946,4977.333333,2648,441.333333,6,0,0,0,0,0,0,0,a,a,1700.0,1.0,2008.0,1,1,14.0,2011.0,"Jan,Apr,Jul,Oct",1,1
6796,51,2013-11-24,47,11,2013,6,38791,13.587040,6465.166667,2855,475.833333,6,5,1,0,0,0,0,0,a,c,10570.0,7.0,2013.0,1,1,9.0,2011.0,"Jan,Apr,Jul,Oct",1,0
22660,168,2015-03-22,12,3,2015,6,50900,10.492682,8483.333333,4851,808.500000,6,5,1,0,0,0,0,0,a,a,12540.0,NaN,NaN,0,0,NaN,NaN,NaN,0,0
41591,309,2013-03-24,12,3,2013,6,47129,12.146649,7854.833333,3880,646.666667,6,5,1,0,0,0,0,0,d,a,8740.0,NaN,NaN,0,1,37.0,2009.0,"Feb,May,Aug,Nov",1,0


### Handle Missing Values

In [13]:
df.isna().sum()

Store                            0
Date                             0
CW                               0
Month                            0
Year                             0
DayOfWeek                        0
Sales                            0
SalesPerCustomer              4716
SalesPerOpenDay               4710
Customers                        0
CustomersPerOpenDay           4710
Open                             0
Promo                            0
IsPromo                          0
StateHoliday                     0
IsStateHoliday                   0
SchoolHoliday                    0
IsSchoolHoliday                  0
NumStateHoliday                  0
StoreType                        0
Assortment                       0
CompetitionDistance            405
CompetitionOpenSinceMonth    47790
CompetitionOpenSinceYear     47790
IsCompetition                    0
Promo2                           0
Promo2SinceWeek              73440
Promo2SinceYear              73440
PromoInterval       

#### SalesPerCustomer, SalesPerOpenday, CustomersPerOpenday

In [14]:
# As the store were closed, we can fill the nans with 0

# fill nans with 0 for listed columns
columns_to_fill = ['SalesPerCustomer', 'SalesPerOpenDay', 'CustomersPerOpenDay']
df_nans_handeled = df.fillna({col: 0 for col in columns_to_fill})
df_nans_handeled


,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,IsCompetition,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2Member,Promo2Active
0,1,2013-01-06,1,1,2013,6,19340,7.736000,4835.000000,2500,625.000000,4,0,0,a,1,6,1,1,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
1,1,2013-01-13,2,1,2013,6,32952,8.410413,5492.000000,3918,653.000000,6,5,1,0,0,5,1,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
2,1,2013-01-20,3,1,2013,6,25978,7.602575,4329.666667,3417,569.500000,6,0,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
3,1,2013-01-27,4,1,2013,6,33071,8.563180,5511.833333,3862,643.666667,6,5,1,0,0,0,0,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
4,1,2013-02-03,5,2,2013,6,28693,8.057568,4782.166667,3561,593.500000,6,0,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,2015-07-05,27,7,2015,6,48130,16.140174,8021.666667,2982,497.000000,6,5,1,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0
150521,1115,2015-07-12,28,7,2015,6,36233,14.315685,6038.833333,2531,421.833333,6,0,0,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0
150522,1115,2015-07-19,29,7,2015,6,45927,15.023553,7654.500000,3057,509.500000,6,5,1,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0
150523,1115,2015-07-26,30,7,2015,6,35362,14.122204,5893.666667,2504,417.333333,6,0,0,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0


In [15]:
df_nans_handeled.isna().sum()

Store                            0
Date                             0
CW                               0
Month                            0
Year                             0
DayOfWeek                        0
Sales                            0
SalesPerCustomer                 0
SalesPerOpenDay                  0
Customers                        0
CustomersPerOpenDay              0
Open                             0
Promo                            0
IsPromo                          0
StateHoliday                     0
IsStateHoliday                   0
SchoolHoliday                    0
IsSchoolHoliday                  0
NumStateHoliday                  0
StoreType                        0
Assortment                       0
CompetitionDistance            405
CompetitionOpenSinceMonth    47790
CompetitionOpenSinceYear     47790
IsCompetition                    0
Promo2                           0
Promo2SinceWeek              73440
Promo2SinceYear              73440
PromoInterval       

#### CompetitionDistance

In [16]:
# Stores with no CompetitionDistance information
print("Stores with no CompetitionDistance information:", df_nans_handeled[(df_nans_handeled['CompetitionDistance'].isna())]['Store'].unique())

print("StoreType of store 291", df_nans_handeled[(df_nans_handeled['Store'] == 291)]['StoreType'].unique())
print("StoreType of store 622", df_nans_handeled[(df_nans_handeled['Store'] == 622)]['StoreType'].unique())
print("StoreType of store 879", df_nans_handeled[(df_nans_handeled['Store'] == 879)]['StoreType'].unique())

Stores with no CompetitionDistance information: [291 622 879]
StoreType of store 291 ['d']
StoreType of store 622 ['a']
StoreType of store 879 ['d']


In [17]:
# As store 291, 622 and 879 have no CompetitionDistance information, we can fill them with the median value of the column

# median competition distance for store type a
median_competition_distance_a = df_nans_handeled[(df_nans_handeled['StoreType'] == 'a')]['CompetitionDistance'].median()
# median competition distance for store type d
median_competition_distance_d = df_nans_handeled[(df_nans_handeled['StoreType'] == 'd')]['CompetitionDistance'].median()

# fill nans for storetype a with median_competition_distance_a
df_nans_handeled.loc[(df_nans_handeled['Store'] == 291), 'CompetitionDistance'] = median_competition_distance_a
# fill nans for storetype d with median_competition_distance_d
df_nans_handeled.loc[(df_nans_handeled['Store'] == 622), 'CompetitionDistance'] = median_competition_distance_d
# fill nans for storetype d with median_competition_distance_d
df_nans_handeled.loc[(df_nans_handeled['Store'] == 879), 'CompetitionDistance'] = median_competition_distance_d



In [18]:
df_nans_handeled.isna().sum()

Store                            0
Date                             0
CW                               0
Month                            0
Year                             0
DayOfWeek                        0
Sales                            0
SalesPerCustomer                 0
SalesPerOpenDay                  0
Customers                        0
CustomersPerOpenDay              0
Open                             0
Promo                            0
IsPromo                          0
StateHoliday                     0
IsStateHoliday                   0
SchoolHoliday                    0
IsSchoolHoliday                  0
NumStateHoliday                  0
StoreType                        0
Assortment                       0
CompetitionDistance              0
CompetitionOpenSinceMonth    47790
CompetitionOpenSinceYear     47790
IsCompetition                    0
Promo2                           0
Promo2SinceWeek              73440
Promo2SinceYear              73440
PromoInterval       

#### CompetitionOpenSinceMonth, CompetitionOpenSinceYear

In [19]:
# CompetitionOpenSinceMonth and CompetitionOpenSinceYear can be deleted as they are reflected in IsCompetition
df_nans_handeled = df_nans_handeled.drop(columns=['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'])

In [20]:
df_nans_handeled.isna().sum()

Store                      0
Date                       0
CW                         0
Month                      0
Year                       0
DayOfWeek                  0
Sales                      0
SalesPerCustomer           0
SalesPerOpenDay            0
Customers                  0
CustomersPerOpenDay        0
Open                       0
Promo                      0
IsPromo                    0
StateHoliday               0
IsStateHoliday             0
SchoolHoliday              0
IsSchoolHoliday            0
NumStateHoliday            0
StoreType                  0
Assortment                 0
CompetitionDistance        0
IsCompetition              0
Promo2                     0
Promo2SinceWeek        73440
Promo2SinceYear        73440
PromoInterval          73440
Promo2Member               0
Promo2Active               0
dtype: int64

#### Promo2SinceWeek, Promo2SinceYear

In [21]:
# Promo2SinceWeek and Promo2SinceYear can be deleted as they are reflected in Promo2Member
df_nans_handeled = df_nans_handeled.drop(columns=['Promo2SinceWeek', 'Promo2SinceYear'])

In [22]:
df_nans_handeled.isna().sum()

Store                      0
Date                       0
CW                         0
Month                      0
Year                       0
DayOfWeek                  0
Sales                      0
SalesPerCustomer           0
SalesPerOpenDay            0
Customers                  0
CustomersPerOpenDay        0
Open                       0
Promo                      0
IsPromo                    0
StateHoliday               0
IsStateHoliday             0
SchoolHoliday              0
IsSchoolHoliday            0
NumStateHoliday            0
StoreType                  0
Assortment                 0
CompetitionDistance        0
IsCompetition              0
Promo2                     0
PromoInterval          73440
Promo2Member               0
Promo2Active               0
dtype: int64

#### PromoInterval

In [23]:
df_nans_handeled[(df_nans_handeled['Promo2'] == 1) & (df_nans_handeled['PromoInterval'].isna())]

,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,IsCompetition,Promo2,PromoInterval,Promo2Member,Promo2Active


In [24]:
# As if the store is not participating in Promo2, PromoInterval is 0, we can fill the nans with 0
df_nans_handeled['PromoInterval'] = df_nans_handeled['PromoInterval'].fillna(0)

In [25]:
df_nans_handeled.isna().sum()

Store                  0
Date                   0
CW                     0
Month                  0
Year                   0
DayOfWeek              0
Sales                  0
SalesPerCustomer       0
SalesPerOpenDay        0
Customers              0
CustomersPerOpenDay    0
Open                   0
Promo                  0
IsPromo                0
StateHoliday           0
IsStateHoliday         0
SchoolHoliday          0
IsSchoolHoliday        0
NumStateHoliday        0
StoreType              0
Assortment             0
CompetitionDistance    0
IsCompetition          0
Promo2                 0
PromoInterval          0
Promo2Member           0
Promo2Active           0
dtype: int64

### Remove not needed Features

In [26]:
# Date is an object and is reflected by CW, Month and Year
df_nans_handeled = df_nans_handeled.drop(columns=['Date'])

# DayOfWeek is not relevant in weekly data
df_nans_handeled = df_nans_handeled.drop(columns=['DayOfWeek'])

df_nans_handeled

,Store,CW,Month,Year,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,IsCompetition,Promo2,PromoInterval,Promo2Member,Promo2Active
0,1,1,1,2013,19340,7.736000,4835.000000,2500,625.000000,4,0,0,a,1,6,1,1,c,a,1270.0,1,0,0,0,0
1,1,2,1,2013,32952,8.410413,5492.000000,3918,653.000000,6,5,1,0,0,5,1,0,c,a,1270.0,1,0,0,0,0
2,1,3,1,2013,25978,7.602575,4329.666667,3417,569.500000,6,0,0,0,0,0,0,0,c,a,1270.0,1,0,0,0,0
3,1,4,1,2013,33071,8.563180,5511.833333,3862,643.666667,6,5,1,0,0,0,0,0,c,a,1270.0,1,0,0,0,0
4,1,5,2,2013,28693,8.057568,4782.166667,3561,593.500000,6,0,0,0,0,0,0,0,c,a,1270.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,27,7,2015,48130,16.140174,8021.666667,2982,497.000000,6,5,1,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0
150521,1115,28,7,2015,36233,14.315685,6038.833333,2531,421.833333,6,0,0,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0
150522,1115,29,7,2015,45927,15.023553,7654.500000,3057,509.500000,6,5,1,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0
150523,1115,30,7,2015,35362,14.122204,5893.666667,2504,417.333333,6,0,0,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0


### Categorical Feature Encoding

In [28]:
df_nans_handeled.select_dtypes(include='object').columns

Index(['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval'], dtype='object')

In [29]:
# check if a column contains mixed data typesrom pandas.api.types import infer_dtype

for col in ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']:
    dtype = infer_dtype(df_nans_handeled[col])
    print(f"Data type of {col}: {dtype}")


Data type of StateHoliday: string
Data type of StoreType: string
Data type of Assortment: string
Data type of PromoInterval: mixed-integer


In [30]:
# Convert mixed columns
cols_to_convert = ['PromoInterval']
df_nans_handeled[cols_to_convert] = df_nans_handeled[cols_to_convert].astype(str)

In [31]:
from sklearn.preprocessing import OneHotEncoder
# handle_unknown='ignore': to avoid error if the training data contains classes/categories that are not represented in the training data
# sparse=False: ensures that the encoded columns are returned as a NumPy array (instead of a sparse matrix).
OneHotEnc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# it is important to pass only the categorical columns, not the whole dataframe
encoded_array = OneHotEnc.fit_transform(df_nans_handeled[['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']])
tmp_cat = pd.DataFrame(encoded_array, columns=OneHotEnc.get_feature_names_out(), index=df_nans_handeled.index)
df_nans_handeled_cat = pd.concat([df_nans_handeled.select_dtypes(include=['number']), tmp_cat], axis=1)
df_nans_handeled_cat

,Store,CW,Month,Year,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,CompetitionDistance,IsCompetition,Promo2,Promo2Member,Promo2Active,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
0,1,1,1,2013,19340,7.736000,4835.000000,2500,625.000000,4,0,0,1,6,1,1,1270.0,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2,1,2013,32952,8.410413,5492.000000,3918,653.000000,6,5,1,0,5,1,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,3,1,2013,25978,7.602575,4329.666667,3417,569.500000,6,0,0,0,0,0,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,4,1,2013,33071,8.563180,5511.833333,3862,643.666667,6,5,1,0,0,0,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,5,2,2013,28693,8.057568,4782.166667,3561,593.500000,6,0,0,0,0,0,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,27,7,2015,48130,16.140174,8021.666667,2982,497.000000,6,5,1,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
150521,1115,28,7,2015,36233,14.315685,6038.833333,2531,421.833333,6,0,0,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
150522,1115,29,7,2015,45927,15.023553,7654.500000,3057,509.500000,6,5,1,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
150523,1115,30,7,2015,35362,14.122204,5893.666667,2504,417.333333,6,0,0,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [32]:
# Creating Lag-Features for these columns
lag_columns = ['Sales', 'SalesPerCustomer', 'SalesPerOpenDay', 'Customers', 'CustomersPerOpenDay']
n_lags = 8  # Anzahl der zu erstellenden Lag-Features
n_periods_for_ma = 4  # amount of periods for moving average
n_periods_for_ma2 = 6
n_periods_for_ma3 = 8

lag_features = []
for col in lag_columns:
    store_groups = df_nans_handeled_cat.groupby('Store')[col]
    for lag in range(1, n_lags + 1):
        lag_col_name = f'{col}_Lag_{lag}'
        # create the lag feature
        lag_feature = store_groups.shift(lag).rename(lag_col_name)
        lag_features.append(lag_feature)

        # create the moving average feature
        ma_col_name = f'{lag_col_name}_MA_{n_periods_for_ma}'
        ma_feature = lag_feature.rolling(window=n_periods_for_ma).mean().rename(ma_col_name)
        lag_features.append(ma_feature)

        ma2_col_name = f'{lag_col_name}_MA_{n_periods_for_ma2}'
        ma2_feature = lag_feature.rolling(window=n_periods_for_ma2).mean().rename(ma2_col_name)
        lag_features.append(ma2_feature)

        ma3_col_name = f'{lag_col_name}_MA_{n_periods_for_ma3}'
        ma3_feature = lag_feature.rolling(window=n_periods_for_ma3).mean().rename(ma3_col_name)
        lag_features.append(ma3_feature)

# Concatenate the lag features
features_df = pd.concat(lag_features, axis=1)
df_nans_handeled_cat = pd.concat([df_nans_handeled_cat, features_df], axis=1)

# Add the future sales
future_sales = df_nans_handeled_cat.groupby('Store')['Sales'].shift(-8).rename('Future_Sales')
df_nans_handeled_cat = pd.concat([df_nans_handeled_cat, future_sales], axis=1)

# Remove rows with NaN values that were created by the shifting and remove the original columns
df_nans_handeled_cat = df_nans_handeled_cat.dropna().drop(columns=lag_columns)
df_nans_handeled_cat


,Store,CW,Month,Year,Open,Promo,IsPromo,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,CompetitionDistance,IsCompetition,Promo2,Promo2Member,Promo2Active,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",Sales_Lag_1,Sales_Lag_1_MA_4,Sales_Lag_1_MA_6,Sales_Lag_1_MA_8,Sales_Lag_2,Sales_Lag_2_MA_4,Sales_Lag_2_MA_6,Sales_Lag_2_MA_8,Sales_Lag_3,Sales_Lag_3_MA_4,Sales_Lag_3_MA_6,Sales_Lag_3_MA_8,Sales_Lag_4,Sales_Lag_4_MA_4,Sales_Lag_4_MA_6,Sales_Lag_4_MA_8,Sales_Lag_5,Sales_Lag_5_MA_4,Sales_Lag_5_MA_6,Sales_Lag_5_MA_8,Sales_Lag_6,Sales_Lag_6_MA_4,Sales_Lag_6_MA_6,Sales_Lag_6_MA_8,Sales_Lag_7,Sales_Lag_7_MA_4,Sales_Lag_7_MA_6,Sales_Lag_7_MA_8,Sales_Lag_8,Sales_Lag_8_MA_4,Sales_Lag_8_MA_6,Sales_Lag_8_MA_8,SalesPerCustomer_Lag_1,SalesPerCustomer_Lag_1_MA_4,SalesPerCustomer_Lag_1_MA_6,SalesPerCustomer_Lag_1_MA_8,SalesPerCustomer_Lag_2,SalesPerCustomer_Lag_2_MA_4,SalesPerCustomer_Lag_2_MA_6,SalesPerCustomer_Lag_2_MA_8,SalesPerCustomer_Lag_3,SalesPerCustomer_Lag_3_MA_4,SalesPerCustomer_Lag_3_MA_6,SalesPerCustomer_Lag_3_MA_8,SalesPerCustomer_Lag_4,SalesPerCustomer_Lag_4_MA_4,SalesPerCustomer_Lag_4_MA_6,SalesPerCustomer_Lag_4_MA_8,SalesPerCustomer_Lag_5,SalesPerCustomer_Lag_5_MA_4,SalesPerCustomer_Lag_5_MA_6,SalesPerCustomer_Lag_5_MA_8,SalesPerCustomer_Lag_6,SalesPerCustomer_Lag_6_MA_4,SalesPerCustomer_Lag_6_MA_6,SalesPerCustomer_Lag_6_MA_8,SalesPerCustomer_Lag_7,SalesPerCustomer_Lag_7_MA_4,SalesPerCustomer_Lag_7_MA_6,SalesPerCustomer_Lag_7_MA_8,SalesPerCustomer_Lag_8,SalesPerCustomer_Lag_8_MA_4,SalesPerCustomer_Lag_8_MA_6,SalesPerCustomer_Lag_8_MA_8,SalesPerOpenDay_Lag_1,SalesPerOpenDay_Lag_1_MA_4,SalesPerOpenDay_Lag_1_MA_6,SalesPerOpenDay_Lag_1_MA_8,SalesPerOpenDay_Lag_2,SalesPerOpenDay_Lag_2_MA_4,SalesPerOpenDay_Lag_2_MA_6,SalesPerOpenDay_Lag_2_MA_8,SalesPerOpenDay_Lag_3,SalesPerOpenDay_Lag_3_MA_4,SalesPerOpenDay_Lag_3_MA_6,SalesPerOpenDay_Lag_3_MA_8,SalesPerOpenDay_Lag_4,SalesPerOpenDay_Lag_4_MA_4,SalesPerOpenDay_Lag_4_MA_6,SalesPerOpenDay_Lag_4_MA_8,SalesPerOpenDay_Lag_5,SalesPerOpenDay_Lag_5_MA_4,SalesPerOpenDay_Lag_5_MA_6,SalesPerOpenDay_Lag_5_MA_8,SalesPerOpenDay_Lag_6,SalesPerOpenDay_Lag_6_MA_4,SalesPerOpenDay_Lag_6_MA_6,SalesPerOpenDay_Lag_6_MA_8,SalesPerOpenDay_Lag_7,SalesPerOpenDay_Lag_7_MA_4,SalesPerOpenDay_Lag_7_MA_6,SalesPerOpenDay_Lag_7_MA_8,SalesPerOpenDay_Lag_8,SalesPerOpenDay_Lag_8_MA_4,SalesPerOpenDay_Lag_8_MA_6,SalesPerOpenDay_Lag_8_MA_8,Customers_Lag_1,Customers_Lag_1_MA_4,Customers_Lag_1_MA_6,Customers_Lag_1_MA_8,Customers_Lag_2,Customers_Lag_2_MA_4,Customers_Lag_2_MA_6,Customers_Lag_2_MA_8,Customers_Lag_3,Customers_Lag_3_MA_4,Customers_Lag_3_MA_6,Customers_Lag_3_MA_8,Customers_Lag_4,Customers_Lag_4_MA_4,Customers_Lag_4_MA_6,Customers_Lag_4_MA_8,Customers_Lag_5,Customers_Lag_5_MA_4,Customers_Lag_5_MA_6,Customers_Lag_5_MA_8,Customers_Lag_6,Customers_Lag_6_MA_4,Customers_Lag_6_MA_6,Customers_Lag_6_MA_8,Customers_Lag_7,Customers_Lag_7_MA_4,Customers_Lag_7_MA_6,Customers_Lag_7_MA_8,Customers_Lag_8,Customers_Lag_8_MA_4,Customers_Lag_8_MA_6,Customers_Lag_8_MA_8,CustomersPerOpenDay_Lag_1,CustomersPerOpenDay_Lag_1_MA_4,CustomersPerOpenDay_Lag_1_MA_6,CustomersPerOpenDay_Lag_1_MA_8,CustomersPerOpenDay_Lag_2,CustomersPerOpenDay_Lag_2_MA_4,CustomersPerOpenDay_Lag_2_MA_6,CustomersPerOpenDay_Lag_2_MA_8,CustomersPerOpenDay_Lag_3,CustomersPerOpenDay_Lag_3_MA_4,CustomersPerOpenDay_Lag_3_MA_6,CustomersPerOpenDay_Lag_3_MA_8,CustomersPerOpenDay_Lag_4,CustomersPerOpenDay_Lag_4_MA_4,CustomersPerOpenDay_Lag_4_MA_6,CustomersPerOpenDay_Lag_4_MA_8,CustomersPerOpenDay_Lag_5,CustomersPerOpenDay_Lag_5_MA_4,CustomersPerOpenDay_Lag_5_MA_6,CustomersPerOpenDay_Lag_5_MA_8,CustomersPerOpenDay_Lag_6,CustomersPerOpenDay_Lag_6_MA_4,CustomersPerOpenDay_Lag_6_MA_6,CustomersPerOpenDay_Lag_6_MA_8,CustomersPerOpenDay_Lag_7,CustomersPerOpen

### New additional Features

#### Ploynominal Features

In [82]:
# Before
testModelsTestSplit8W(df_nans_handeled_cat, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9349.951020349768, 'MAE_Train': 6453.03660119806, 'R2_Train': 0.7366461886920278, 'Adj_R2_Train': 0.7362116940306376, 'RMSE_Test': 6031.943678923188, 'MAE_Test': 4500.710893128057, 'R2_Test': 0.8599439761981675, 'Adj_R2_Test': 0.8568790471713401}
{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980693987589, 'MAE_Train': 5030.000077158206, 'R2_Train': 0.8345478937990809, 'Adj_R2_Train': 0.8342749224055666, 'RMSE_Test': 6058.149998380618, 'MAE_Test': 4515.475634765625, 'R2_Test': 0.8587243607024304, 'Adj_R2_Test': 0.8556327421064364}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9349.951020,6453.036601,0.736646,0.736212,6031.943679,4500.710893,0.859944,0.856879
1,XGBRegressor,7410.980694,5030.000077,0.834548,0.834275,6058.149998,4515.475635,0.858724,0.855633


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2)
features = ['Open', 'Promo', 'IsPromo', 'IsStateHoliday', 'SchoolHoliday', 'IsSchoolHoliday', 'CompetitionDistance', 'IsCompetition', 'Promo2Member', 'Assortment_a', 'Assortment_b', 'Assortment_c']

feat_array = pf.fit_transform(df_nans_handeled_cat[features])
# generate names for the new features
feature_names = pf.get_feature_names_out(input_features=features)
poly_features_df = pd.DataFrame(feat_array, columns=feature_names, index=df_nans_handeled_cat.index)
# remove the '1' column
poly_features_df = poly_features_df.drop('1', axis=1)
# Concatenate the polynomial features with the original DataFrame
df_nans_handeled_cat_poly = pd.concat([df_nans_handeled_cat, poly_features_df], axis=1)
testModelsTestSplit8W(df_nans_handeled_cat_poly, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9196.055248192568, 'MAE_Train': 6324.875376361263, 'R2_Train': 0.745244200308042, 'Adj_R2_Train': 0.7446253585255644, 'RMSE_Test': 6753.577811387857, 'MAE_Test': 5219.836299709872, 'R2_Test': 0.8244280822823356, 'Adj_R2_Test': 0.8187166086913812}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9196.055248,6324.875376,0.745244,0.744625,6753.577811,5219.8363,0.824428,0.818717


**Results:**
- Before Polynominal Features: MAE: 4495.405879	R2: 0.860075
- After Polynominal Features: MAE: 5219.8363	R2: 0.824428

-> Polynominal Features did not improve the model and will not be used

## Skewness

In [33]:
#pd.set_option('display.max_rows', None)
num_columns_float = df_nans_handeled_cat.select_dtypes(include='number').columns
skew = df_nans_handeled_cat[num_columns_float].skew().sort_values(ascending=False)
skew[skew >= 3]

Assortment_b            10.995450
StoreType_b              7.912346
StateHoliday_c           7.611515
StateHoliday_b           5.003762
Customers_Lag_1_MA_8     3.079217
Customers_Lag_2_MA_8     3.073890
Customers_Lag_3_MA_8     3.068401
Customers_Lag_1_MA_6     3.065142
Customers_Lag_4_MA_8     3.063038
Customers_Lag_2_MA_6     3.060226
Customers_Lag_5_MA_8     3.059181
Customers_Lag_3_MA_6     3.055243
Customers_Lag_6_MA_8     3.054436
Customers_Lag_7_MA_8     3.050417
Customers_Lag_1_MA_4     3.049608
Customers_Lag_4_MA_6     3.049238
Customers_Lag_8_MA_8     3.046789
Customers_Lag_2_MA_4     3.046094
Customers_Lag_5_MA_6     3.044501
Customers_Lag_3_MA_4     3.040599
Customers_Lag_6_MA_6     3.038744
Customers_Lag_4_MA_4     3.034761
Customers_Lag_7_MA_6     3.034684
Customers_Lag_5_MA_4     3.031263
Customers_Lag_8_MA_6     3.030890
Customers_Lag_6_MA_4     3.024336
Customers_Lag_7_MA_4     3.018899
Customers_Lag_8_MA_4     3.014113
dtype: float64

In [35]:
# Before
testModelsTestSplit8W(df_nans_handeled_cat, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9348.624262728845, 'MAE_Train': 6448.18666177155, 'R2_Train': 0.7367209231767318, 'Adj_R2_Train': 0.7362865518161379, 'RMSE_Test': 6029.131809472689, 'MAE_Test': 4495.405879127806, 'R2_Test': 0.8600745236580398, 'Adj_R2_Test': 0.8570124514786959}


c:\Users\Chris\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [21:45:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980873242441, 'MAE_Train': 5030.000189809052, 'R2_Train': 0.8345478857952568, 'Adj_R2_Train': 0.8342749143885373, 'RMSE_Test': 6058.148984876124, 'MAE_Test': 4515.474892052621, 'R2_Test': 0.858724407972136, 'Adj_R2_Test': 0.8556327904105729}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9348.624263,6448.186662,0.736721,0.736287,6029.131809,4495.405879,0.860075,0.857012
1,XGBRegressor,7410.980873,5030.000190,0.834548,0.834275,6058.148985,4515.474892,0.858724,0.855633


In [85]:
# Log transformation
df_nans_handeled_cat_log = df_nans_handeled_cat.copy()
for_log_transform = skew[skew >= 3].index
df_nans_handeled_cat_log[for_log_transform] = np.log(df_nans_handeled_cat_log[for_log_transform]+1)
testModelsTestSplit8W(df_nans_handeled_cat_log, None)

{'Model': 'LinearRegression', 'RMSE_Train': 8996.267242713275, 'MAE_Train': 6127.220069480263, 'R2_Train': 0.7561933021058949, 'Adj_R2_Train': 0.7557910572774641, 'RMSE_Test': 5895.625910323417, 'MAE_Test': 4370.480975307157, 'R2_Test': 0.8662027847087163, 'Adj_R2_Test': 0.8632748209002108}
{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980693987589, 'MAE_Train': 5030.000077158206, 'R2_Train': 0.8345478937990809, 'Adj_R2_Train': 0.8342749224055666, 'RMSE_Test': 6058.149998380618, 'MAE_Test': 4515.475634765625, 'R2_Test': 0.8587243607024304, 'Adj_R2_Test': 0.8556327421064364}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,8996.267243,6127.220069,0.756193,0.755791,5895.625910,4370.480975,0.866203,0.863275
1,XGBRegressor,7410.980694,5030.000077,0.834548,0.834275,6058.149998,4515.475635,0.858724,0.855633


In [36]:
# use PowerTransformer to transform the data
df_nans_handeled_cat_power = df_nans_handeled_cat.copy()
for_log_transform = skew[skew >= 3].index
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()
df_nans_handeled_cat_power[for_log_transform] = pt.fit_transform(df_nans_handeled_cat_power[for_log_transform])
testModelsTestSplit8W(df_nans_handeled_cat_power, None)


{'Model': 'LinearRegression', 'RMSE_Train': 9080.95770607921, 'MAE_Train': 6230.562682192093, 'R2_Train': 0.7515813243113636, 'Adj_R2_Train': 0.7511714704047873, 'RMSE_Test': 5959.227147139775, 'MAE_Test': 4414.332558622488, 'R2_Test': 0.8633004401241766, 'Adj_R2_Test': 0.860308962587939}


c:\Users\Chris\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [21:46:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980873242441, 'MAE_Train': 5030.000189809052, 'R2_Train': 0.8345478857952568, 'Adj_R2_Train': 0.8342749143885373, 'RMSE_Test': 6058.148984876124, 'MAE_Test': 4515.474892052621, 'R2_Test': 0.858724407972136, 'Adj_R2_Test': 0.8556327904105729}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9080.957706,6230.562682,0.751581,0.751171,5959.227147,4414.332559,0.863300,0.860309
1,XGBRegressor,7410.980873,5030.000190,0.834548,0.834275,6058.148985,4515.474892,0.858724,0.855633


**Results:**
- Before: MAE:4793.96383	R2:0.844227
- After Power Transformation skew>=2: MAE:4615.821447	R2:	0.850871
- After Power Transformation skew>=3: MAE:4682.241185	R2: 0.847547
- After log transformation skew>=2: MAE: 4641.626858	R2: 0.851209
- After log transformation skew>=3: MAE: 4528.091135	R2: 0.854112

-> Log transformation with skew>=3 will be used

After adding ma3:
- Before: MAE: 4495.405879	R2: 0.860075
- After log transformation skew>=3: MAE: 4512.055025	R2:0.857954  
- After Power transformation skew>=3: MAE: 4414.332559	R2: 0.8633

--> Power transformation with skew>=3 will be used

## Feature Scaling

In [87]:
# Before
testModelsTestSplit8W(df_nans_handeled_cat_power, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9080.722238731532, 'MAE_Train': 6231.216114108374, 'R2_Train': 0.7515942070336924, 'Adj_R2_Train': 0.7511843743816939, 'RMSE_Test': 5962.597616137577, 'MAE_Test': 4418.725909494953, 'R2_Test': 0.8631457650590293, 'Adj_R2_Test': 0.8601509026766134}
{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980693987589, 'MAE_Train': 5030.000077158206, 'R2_Train': 0.8345478937990809, 'Adj_R2_Train': 0.8342749224055666, 'RMSE_Test': 6058.149998380618, 'MAE_Test': 4515.475634765625, 'R2_Test': 0.8587243607024304, 'Adj_R2_Test': 0.8556327421064364}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9080.722239,6231.216114,0.751594,0.751184,5962.597616,4418.725909,0.863146,0.860151
1,XGBRegressor,7410.980694,5030.000077,0.834548,0.834275,6058.149998,4515.475635,0.858724,0.855633


### StandardScaler

In [88]:
testModelsTestSplit8W(df_nans_handeled_cat_power, StandardScaler())

{'Model': 'LinearRegression', 'RMSE_Train': 9080.673704354314, 'MAE_Train': 6230.754311109099, 'R2_Train': 0.7515968623705528, 'Adj_R2_Train': 0.7511870340994656, 'RMSE_Test': 5960.074585783354, 'MAE_Test': 4417.419570876309, 'R2_Test': 0.8632615583283878, 'Adj_R2_Test': 0.8602692299187547}
{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980693987589, 'MAE_Train': 5030.000077158206, 'R2_Train': 0.8345478937990809, 'Adj_R2_Train': 0.8342749224055666, 'RMSE_Test': 6058.149998380618, 'MAE_Test': 4515.475634765625, 'R2_Test': 0.8587243607024304, 'Adj_R2_Test': 0.8556327421064364}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9080.673704,6230.754311,0.751597,0.751187,5960.074586,4417.419571,0.863262,0.860269
1,XGBRegressor,7410.980694,5030.000077,0.834548,0.834275,6058.149998,4515.475635,0.858724,0.855633


### MinMaxScaler

In [89]:
testModelsTestSplit8W(df_nans_handeled_cat_power, MinMaxScaler())

{'Model': 'LinearRegression', 'RMSE_Train': 9080.879286303141, 'MAE_Train': 6231.961943773715, 'R2_Train': 0.751585614795569, 'Adj_R2_Train': 0.7511757679676542, 'RMSE_Test': 5965.482629551052, 'MAE_Test': 4419.235538116592, 'R2_Test': 0.8630132986919783, 'Adj_R2_Test': 0.8600155374694952}
{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980693987589, 'MAE_Train': 5030.000077158206, 'R2_Train': 0.8345478937990809, 'Adj_R2_Train': 0.8342749224055666, 'RMSE_Test': 6058.149998380618, 'MAE_Test': 4515.475634765625, 'R2_Test': 0.8587243607024304, 'Adj_R2_Test': 0.8556327421064364}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9080.879286,6231.961944,0.751586,0.751176,5965.482630,4419.235538,0.863013,0.860016
1,XGBRegressor,7410.980694,5030.000077,0.834548,0.834275,6058.149998,4515.475635,0.858724,0.855633


### RobustScaler

In [90]:
testModelsTestSplit8W(df_nans_handeled_cat_power, RobustScaler())

{'Model': 'LinearRegression', 'RMSE_Train': 9080.67444153205, 'MAE_Train': 6230.54950333089, 'R2_Train': 0.7515968220393449, 'Adj_R2_Train': 0.7511869937017172, 'RMSE_Test': 5959.1882336524495, 'MAE_Test': 4416.791974670684, 'R2_Test': 0.8633022254023986, 'Adj_R2_Test': 0.8603107869344631}
{'Model': 'XGBRegressor', 'RMSE_Train': 7410.980693987589, 'MAE_Train': 5030.000077158206, 'R2_Train': 0.8345478937990809, 'Adj_R2_Train': 0.8342749224055666, 'RMSE_Test': 6058.149998380618, 'MAE_Test': 4515.475634765625, 'R2_Test': 0.8587243607024304, 'Adj_R2_Test': 0.8556327421064364}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9080.674442,6230.549503,0.751597,0.751187,5959.188234,4416.791975,0.863302,0.860311
1,XGBRegressor,7410.980694,5030.000077,0.834548,0.834275,6058.149998,4515.475635,0.858724,0.855633


**Result: StandardScaler, MinMaxScaler, RobustScaler have just a small effect on the model performance.**
- Before at LinearRegression: MAE: 4414.332559	R2: 0.8633
- After MinMaxScaler at LinearRegression: MAE: 4398.551738	R2: 0.864094

-> MinMaxScaler will be used

In [ ]:
# Save df as csv
df_nans_handeled_cat_power.to_csv('df_nans_handeled_cat_power.csv', index=False)

In [43]:
df_nans_handeled_cat_power

,Store,CW,Month,Year,Open,Promo,IsPromo,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,CompetitionDistance,IsCompetition,Promo2,Promo2Member,Promo2Active,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",Sales_Lag_1,Sales_Lag_1_MA_4,Sales_Lag_1_MA_6,Sales_Lag_1_MA_8,Sales_Lag_2,Sales_Lag_2_MA_4,Sales_Lag_2_MA_6,Sales_Lag_2_MA_8,Sales_Lag_3,Sales_Lag_3_MA_4,Sales_Lag_3_MA_6,Sales_Lag_3_MA_8,Sales_Lag_4,Sales_Lag_4_MA_4,Sales_Lag_4_MA_6,Sales_Lag_4_MA_8,Sales_Lag_5,Sales_Lag_5_MA_4,Sales_Lag_5_MA_6,Sales_Lag_5_MA_8,Sales_Lag_6,Sales_Lag_6_MA_4,Sales_Lag_6_MA_6,Sales_Lag_6_MA_8,Sales_Lag_7,Sales_Lag_7_MA_4,Sales_Lag_7_MA_6,Sales_Lag_7_MA_8,Sales_Lag_8,Sales_Lag_8_MA_4,Sales_Lag_8_MA_6,Sales_Lag_8_MA_8,SalesPerCustomer_Lag_1,SalesPerCustomer_Lag_1_MA_4,SalesPerCustomer_Lag_1_MA_6,SalesPerCustomer_Lag_1_MA_8,SalesPerCustomer_Lag_2,SalesPerCustomer_Lag_2_MA_4,SalesPerCustomer_Lag_2_MA_6,SalesPerCustomer_Lag_2_MA_8,SalesPerCustomer_Lag_3,SalesPerCustomer_Lag_3_MA_4,SalesPerCustomer_Lag_3_MA_6,SalesPerCustomer_Lag_3_MA_8,SalesPerCustomer_Lag_4,SalesPerCustomer_Lag_4_MA_4,SalesPerCustomer_Lag_4_MA_6,SalesPerCustomer_Lag_4_MA_8,SalesPerCustomer_Lag_5,SalesPerCustomer_Lag_5_MA_4,SalesPerCustomer_Lag_5_MA_6,SalesPerCustomer_Lag_5_MA_8,SalesPerCustomer_Lag_6,SalesPerCustomer_Lag_6_MA_4,SalesPerCustomer_Lag_6_MA_6,SalesPerCustomer_Lag_6_MA_8,SalesPerCustomer_Lag_7,SalesPerCustomer_Lag_7_MA_4,SalesPerCustomer_Lag_7_MA_6,SalesPerCustomer_Lag_7_MA_8,SalesPerCustomer_Lag_8,SalesPerCustomer_Lag_8_MA_4,SalesPerCustomer_Lag_8_MA_6,SalesPerCustomer_Lag_8_MA_8,SalesPerOpenDay_Lag_1,SalesPerOpenDay_Lag_1_MA_4,SalesPerOpenDay_Lag_1_MA_6,SalesPerOpenDay_Lag_1_MA_8,SalesPerOpenDay_Lag_2,SalesPerOpenDay_Lag_2_MA_4,SalesPerOpenDay_Lag_2_MA_6,SalesPerOpenDay_Lag_2_MA_8,SalesPerOpenDay_Lag_3,SalesPerOpenDay_Lag_3_MA_4,SalesPerOpenDay_Lag_3_MA_6,SalesPerOpenDay_Lag_3_MA_8,SalesPerOpenDay_Lag_4,SalesPerOpenDay_Lag_4_MA_4,SalesPerOpenDay_Lag_4_MA_6,SalesPerOpenDay_Lag_4_MA_8,SalesPerOpenDay_Lag_5,SalesPerOpenDay_Lag_5_MA_4,SalesPerOpenDay_Lag_5_MA_6,SalesPerOpenDay_Lag_5_MA_8,SalesPerOpenDay_Lag_6,SalesPerOpenDay_Lag_6_MA_4,SalesPerOpenDay_Lag_6_MA_6,SalesPerOpenDay_Lag_6_MA_8,SalesPerOpenDay_Lag_7,SalesPerOpenDay_Lag_7_MA_4,SalesPerOpenDay_Lag_7_MA_6,SalesPerOpenDay_Lag_7_MA_8,SalesPerOpenDay_Lag_8,SalesPerOpenDay_Lag_8_MA_4,SalesPerOpenDay_Lag_8_MA_6,SalesPerOpenDay_Lag_8_MA_8,Customers_Lag_1,Customers_Lag_1_MA_4,Customers_Lag_1_MA_6,Customers_Lag_1_MA_8,Customers_Lag_2,Customers_Lag_2_MA_4,Customers_Lag_2_MA_6,Customers_Lag_2_MA_8,Customers_Lag_3,Customers_Lag_3_MA_4,Customers_Lag_3_MA_6,Customers_Lag_3_MA_8,Customers_Lag_4,Customers_Lag_4_MA_4,Customers_Lag_4_MA_6,Customers_Lag_4_MA_8,Customers_Lag_5,Customers_Lag_5_MA_4,Customers_Lag_5_MA_6,Customers_Lag_5_MA_8,Customers_Lag_6,Customers_Lag_6_MA_4,Customers_Lag_6_MA_6,Customers_Lag_6_MA_8,Customers_Lag_7,Customers_Lag_7_MA_4,Customers_Lag_7_MA_6,Customers_Lag_7_MA_8,Customers_Lag_8,Customers_Lag_8_MA_4,Customers_Lag_8_MA_6,Customers_Lag_8_MA_8,CustomersPerOpenDay_Lag_1,CustomersPerOpenDay_Lag_1_MA_4,CustomersPerOpenDay_Lag_1_MA_6,CustomersPerOpenDay_Lag_1_MA_8,CustomersPerOpenDay_Lag_2,CustomersPerOpenDay_Lag_2_MA_4,CustomersPerOpenDay_Lag_2_MA_6,CustomersPerOpenDay_Lag_2_MA_8,CustomersPerOpenDay_Lag_3,CustomersPerOpenDay_Lag_3_MA_4,CustomersPerOpenDay_Lag_3_MA_6,CustomersPerOpenDay_Lag_3_MA_8,CustomersPerOpenDay_Lag_4,CustomersPerOpenDay_Lag_4_MA_4,CustomersPerOpenDay_Lag_4_MA_6,CustomersPerOpenDay_Lag_4_MA_8,CustomersPerOpenDay_Lag_5,CustomersPerOpenDay_Lag_5_MA_4,CustomersPerOpenDay_Lag_5_MA_6,CustomersPerOpenDay_Lag_5_MA_8,CustomersPerOpenDay_Lag_6,CustomersPerOpenDay_Lag_6_MA_4,CustomersPerOpenDay_Lag_6_MA_6,CustomersPerOpenDay_Lag_6_MA_8,CustomersPerOpenDay_Lag_7,CustomersPerOpen

## Feature reduction

In [47]:
train_data = []
test_data = []
# Group by store and split into training and test data
amount_test_weeks = 8
for store_id, group in df_nans_handeled_cat_power.groupby('Store'):
	train_data.append(group[: -amount_test_weeks])
	test_data.append(group[-amount_test_weeks:])
# Combine the list entries to one dataframe
train_df = pd.concat(train_data)
test_df = pd.concat(test_data)
X_train = train_df.drop(columns=['Future_Sales'])
y_train = train_df['Future_Sales']
X_test = test_df.drop(columns=['Future_Sales'])
y_test = test_df['Future_Sales']

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit

model = LinearRegression(n_jobs=-1)
tscv = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(model, n_features_to_select=40, direction='forward', cv=tscv)

# perform feature selection
sfs.fit(X_train, y_train)

# get the selected feature indices
selected_features_boolean = sfs.get_support()

# extract selected column names
selected_columns = X_train.columns[selected_features_boolean]

print("Ausgewählte Spaltennamen:", selected_columns)

Ausgewählte Spaltennamen: Index(['CW', 'Month', 'Open', 'Promo', 'IsPromo', 'IsSchoolHoliday',
       'StateHoliday_a', 'StateHoliday_b', 'Assortment_b', 'Assortment_c',
       'Sales_Lag_1', 'Sales_Lag_1_MA_6', 'Sales_Lag_1_MA_8',
       'Sales_Lag_2_MA_8', 'Sales_Lag_3', 'Sales_Lag_3_MA_8',
       'Sales_Lag_4_MA_4', 'Sales_Lag_5_MA_8', 'Sales_Lag_7_MA_8',
       'Sales_Lag_8_MA_8', 'SalesPerCustomer_Lag_3_MA_6',
       'SalesPerCustomer_Lag_4_MA_6', 'SalesPerCustomer_Lag_7',
       'SalesPerCustomer_Lag_8', 'SalesPerOpenDay_Lag_1',
       'SalesPerOpenDay_Lag_4_MA_6', 'SalesPerOpenDay_Lag_5_MA_8',
       'SalesPerOpenDay_Lag_7', 'Customers_Lag_1_MA_4', 'Customers_Lag_1_MA_6',
       'Customers_Lag_3_MA_4', 'Customers_Lag_4', 'Customers_Lag_4_MA_8',
       'Customers_Lag_6', 'Customers_Lag_7_MA_8', 'Customers_Lag_8_MA_8',
       'CustomersPerOpenDay_Lag_6', 'CustomersPerOpenDay_Lag_6_MA_4',
       'CustomersPerOpenDay_Lag_7_MA_4', 'CustomersPerOpenDay_Lag_8'],
      dtype='object')


In [53]:
train_data = []
test_data = []
# Group by store and split into training and test data
amount_test_weeks = 8
df_to_reduce = df_nans_handeled_cat_power[['Store', 'Future_Sales', 'CW', 'Month', 'Open', 'Promo', 'IsPromo', 'IsSchoolHoliday',
       'StateHoliday_a', 'StateHoliday_b', 'Assortment_b', 'Assortment_c',
       'Sales_Lag_1', 'Sales_Lag_1_MA_6', 'Sales_Lag_1_MA_8',
       'Sales_Lag_2_MA_8', 'Sales_Lag_3', 'Sales_Lag_3_MA_8',
       'Sales_Lag_4_MA_4', 'Sales_Lag_5_MA_8', 'Sales_Lag_7_MA_8',
       'Sales_Lag_8_MA_8', 'SalesPerCustomer_Lag_3_MA_6',
       'SalesPerCustomer_Lag_4_MA_6', 'SalesPerCustomer_Lag_7',
       'SalesPerCustomer_Lag_8', 'SalesPerOpenDay_Lag_1',
       'SalesPerOpenDay_Lag_4_MA_6', 'SalesPerOpenDay_Lag_5_MA_8',
       'SalesPerOpenDay_Lag_7', 'Customers_Lag_1_MA_4', 'Customers_Lag_1_MA_6',
       'Customers_Lag_3_MA_4', 'Customers_Lag_4', 'Customers_Lag_4_MA_8',
       'Customers_Lag_6', 'Customers_Lag_7_MA_8', 'Customers_Lag_8_MA_8',
       'CustomersPerOpenDay_Lag_6', 'CustomersPerOpenDay_Lag_6_MA_4',
       'CustomersPerOpenDay_Lag_7_MA_4', 'CustomersPerOpenDay_Lag_8']]
for store_id, group in df_to_reduce.groupby('Store'):
	train_data.append(group[: -amount_test_weeks])
	test_data.append(group[-amount_test_weeks:])
# Combine the list entries to one dataframe
train_df = pd.concat(train_data)
test_df = pd.concat(test_data)
X_train = train_df.drop(columns=['Future_Sales'])
y_train = train_df['Future_Sales']
X_test = test_df.drop(columns=['Future_Sales'])
y_test = test_df['Future_Sales']

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit

model = LinearRegression(n_jobs=-1)
tscv = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(model, n_features_to_select=20, direction='forward', cv=tscv)

# perform feature selection
sfs.fit(X_train, y_train)

# get the selected feature indices
selected_features_boolean = sfs.get_support()

# extract selected column names
selected_columns = X_train.columns[selected_features_boolean]

print("Ausgewählte Spaltennamen:", selected_columns)

Ausgewählte Spaltennamen: Index(['Open', 'Promo', 'IsPromo', 'StateHoliday_b', 'Sales_Lag_1',
       'Sales_Lag_1_MA_6', 'Sales_Lag_1_MA_8', 'Sales_Lag_2_MA_8',
       'Sales_Lag_3_MA_8', 'Sales_Lag_5_MA_8', 'Sales_Lag_8_MA_8',
       'SalesPerCustomer_Lag_3_MA_6', 'SalesPerCustomer_Lag_8',
       'SalesPerOpenDay_Lag_1', 'SalesPerOpenDay_Lag_4_MA_6',
       'Customers_Lag_1_MA_6', 'Customers_Lag_3_MA_4', 'Customers_Lag_6',
       'Customers_Lag_8_MA_8', 'CustomersPerOpenDay_Lag_6'],
      dtype='object')


In [63]:
# with 40 selected features
df_test = df_nans_handeled_cat_power[['Store', 'Future_Sales', 'CW', 'Month', 'Open', 'Promo', 'IsPromo', 'IsSchoolHoliday',
       'StateHoliday_a', 'StateHoliday_b', 'Assortment_b', 'Assortment_c',
       'Sales_Lag_1', 'Sales_Lag_1_MA_6', 'Sales_Lag_1_MA_8',
       'Sales_Lag_2_MA_8', 'Sales_Lag_3', 'Sales_Lag_3_MA_8',
       'Sales_Lag_4_MA_4', 'Sales_Lag_5_MA_8', 'Sales_Lag_7_MA_8',
       'Sales_Lag_8_MA_8', 'SalesPerCustomer_Lag_3_MA_6',
       'SalesPerCustomer_Lag_4_MA_6', 'SalesPerCustomer_Lag_7',
       'SalesPerCustomer_Lag_8', 'SalesPerOpenDay_Lag_1',
       'SalesPerOpenDay_Lag_4_MA_6', 'SalesPerOpenDay_Lag_5_MA_8',
       'SalesPerOpenDay_Lag_7', 'Customers_Lag_1_MA_4', 'Customers_Lag_1_MA_6',
       'Customers_Lag_3_MA_4', 'Customers_Lag_4', 'Customers_Lag_4_MA_8',
       'Customers_Lag_6', 'Customers_Lag_7_MA_8', 'Customers_Lag_8_MA_8',
       'CustomersPerOpenDay_Lag_6', 'CustomersPerOpenDay_Lag_6_MA_4',
       'CustomersPerOpenDay_Lag_7_MA_4', 'CustomersPerOpenDay_Lag_8']]
testModelsTestSplit8W(df_test, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9243.960853576737, 'MAE_Train': 6346.849079059969, 'R2_Train': 0.7425830551488228, 'Adj_R2_Train': 0.7424920072120149, 'RMSE_Test': 6076.866293181645, 'MAE_Test': 4604.182223657593, 'R2_Test': 0.8578500868741831, 'Adj_R2_Test': 0.8571936162233429}


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:25:06] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


{'Model': 'XGBRegressor', 'RMSE_Train': 7586.747425730487, 'MAE_Train': 5128.276978852914, 'R2_Train': 0.8266067494562147, 'Adj_R2_Train': 0.8265454205575768, 'RMSE_Test': 6139.361351649675, 'MAE_Test': 4697.845359576443, 'R2_Test': 0.8549112868641222, 'Adj_R2_Test': 0.854241244372731}
{'Model': 'RandomForestRegressor', 'RMSE_Train': 6833.118635588801, 'MAE_Train': 4610.654046070799, 'R2_Train': 0.8593438071493607, 'Adj_R2_Train': 0.859294057292506, 'RMSE_Test': 6633.261060700181, 'MAE_Test': 5053.971024294538, 'R2_Test': 0.8306280776890549, 'Adj_R2_Test': 0.8298458915193377}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9243.960854,6346.849079,0.742583,0.742492,6076.866293,4604.182224,0.857850,0.857194
1,XGBRegressor,7586.747426,5128.276979,0.826607,0.826545,6139.361352,4697.845360,0.854911,0.854241
2,RandomForestRegressor,6833.118636,4610.654046,0.859344,0.859294,6633.261061,5053.971024,0.830628,0.829846


In [49]:
# with 20 selected features

df_test = df_nans_handeled_cat_power[['Store', 'Future_Sales', 'Open', 'Promo', 'IsPromo', 'StateHoliday_b', 'Sales_Lag_1',
       'Sales_Lag_1_MA_6', 'Sales_Lag_1_MA_8', 'Sales_Lag_2_MA_8',
       'Sales_Lag_3_MA_8', 'Sales_Lag_5_MA_8', 'Sales_Lag_8_MA_8',
       'SalesPerCustomer_Lag_3_MA_6', 'SalesPerCustomer_Lag_8',
       'SalesPerOpenDay_Lag_1', 'SalesPerOpenDay_Lag_4_MA_6',
       'Customers_Lag_1_MA_6', 'Customers_Lag_3_MA_4', 'Customers_Lag_6',
       'Customers_Lag_8_MA_8', 'CustomersPerOpenDay_Lag_6']]
testModelsTestSplit8W(df_test, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9536.329179581322, 'MAE_Train': 6673.629827187907, 'R2_Train': 0.7260423678114407, 'Adj_R2_Train': 0.7259927455109356, 'RMSE_Test': 5775.47628316968, 'MAE_Test': 4295.672175582573, 'R2_Test': 0.8716006437113271, 'Adj_R2_Test': 0.8712976108407874}


c:\Users\Chris\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [22:09:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


{'Model': 'XGBRegressor', 'RMSE_Train': 9457.068947317517, 'MAE_Train': 6669.648039198498, 'R2_Train': 0.7305773851169427, 'Adj_R2_Train': 0.7305285842499919, 'RMSE_Test': 6298.190214583256, 'MAE_Test': 4920.265377574972, 'R2_Test': 0.8473071214064962, 'Adj_R2_Test': 0.8469467538575567}


c:\Users\Chris\anaconda3\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


{'Model': 'NeuralNetwork', 'RMSE_Train': 12397.468610666763, 'MAE_Train': 9263.2136661244, 'R2_Train': 0.5369936247054163, 'Adj_R2_Train': 0.5369097597613843, 'RMSE_Test': 11164.801701721253, 'MAE_Test': 9001.246323877172, 'R2_Test': 0.5201679368692144, 'Adj_R2_Test': 0.5190354943736258}


c:\Users\Chris\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'Model': 'MLPRegressor', 'RMSE_Train': 8739.366559683718, 'MAE_Train': 5797.459515852285, 'R2_Train': 0.7699189517258402, 'Adj_R2_Train': 0.7698772768477695, 'RMSE_Test': 6647.838726860592, 'MAE_Test': 5149.058839719626, 'R2_Test': 0.8298828151753054, 'Adj_R2_Test': 0.8294813248537366}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9536.329180,6673.629827,0.726042,0.725993,5775.476283,4295.672176,0.871601,0.871298
1,XGBRegressor,9457.068947,6669.648039,0.730577,0.730529,6298.190215,4920.265378,0.847307,0.846947
2,NeuralNetwork,12397.468611,9263.213666,0.536994,0.536910,11164.801702,9001.246324,0.520168,0.519035
3,MLPRegressor,8739.366560,5797.459516,0.769919,0.769877,6647.838727,5149.058840,0.829883,0.829481


**Result:**
- LinearRegression increased slightly from R2:0,86 to R2:0,87
- XGBRegressor decreased slightly from R2: 0,859 to R2:0,847

-> The Top 20 will be used